In [ ]:
class Node:
    def __init__(self, keys=None, children=None):
        self.keys = keys or []
        self.children = children or []

    def is_leaf(self):
        return len(self.children) == 0

    def is_full(self):
        # Estamos permitindo até 4 chaves; full é quando tem 5 (precisa split)
        return len(self.keys) > 3

    def is_deficient(self):
        # Para nós que não são raiz, abaixo de 2 é deficiente
        return len(self.keys) < 3

In [ ]:
class StrictBTree:
    def __init__(self):
        self.root = Node()

    ############### SPLIT ###############
    def split_child(self, parent, index):
      child = parent.children[index]

      mid = len(child.keys) // 2
      median = child.keys[mid]

      # criar nós esquerdo e direito
      left = Node(child.keys[:mid], child.children[:mid+1] if not child.is_leaf() else [])
      right = Node(child.keys[mid+1:], child.children[mid+1:] if not child.is_leaf() else [])

      # inserir a mediana no pai
      parent.keys.insert(index, median)

      # atualizar filhos do pai
      parent.children[index] = left
      parent.children.insert(index+1, right)


    ############### INSERTION ###############
    def insert(self, key):
        root = self.root
        # se raiz está cheia (tem >4 chaves), criar nova raiz e dividir
        if root.is_full():
            new_root = Node(children=[root])
            # split_child espera (parent_node, index_of_child)
            self.split_child(new_root, 0)
            self.root = new_root
        else:
          self._insert_non_full(self.root, key)

    def _insert_non_full(self, node, key):
        if node.is_leaf():
            node.keys.append(key)
            node.keys.sort()
            return

        # achar filho apropriado
        i = 0
        while i < len(node.keys) and key > node.keys[i]:
            i += 1

        # se o filho está cheio (tem >4 chaves), dividir antes de descer
        if node.children[i].is_full():
            self.split_child(node, i)
            # após split, a mediana subiu para node.keys[i]; decidir direção
            if key > node.keys[i]:
                i += 1

        self._insert_non_full(node.children[i], key)

    ############### REMOVAL ###############
    def remove(self, key):
        self._remove(self.root, key)
        # se após remoção a raiz ficou vazia (0 chaves) e não é folha, promover filho
        if len(self.root.keys) == 0 and not self.root.is_leaf():
            self.root = self.root.children[0]

    def _remove(self, node, key):
        if node.is_leaf():
            if key in node.keys:
                node.keys.remove(key)
            return

        # encontrar posição
        i = 0
        while i < len(node.keys) and key > node.keys[i]:
            i += 1

        if i < len(node.keys) and node.keys[i] == key:
            return self._remove_internal(node, key, i)
        else:
            return self._remove_child(node, key, i)

    def _remove_internal(self, node, key, index):
        # Caso 1: predecessor suficiente
        if len(node.children[index].keys) >= 3:
            pred = self._get_max(node.children[index])
            node.keys[index] = pred
            self._remove(node.children[index], pred)
        # Caso 2: successor suficiente
        elif len(node.children[index + 1].keys) >= 3:
            succ = self._get_min(node.children[index + 1])
            node.keys[index] = succ
            self._remove(node.children[index + 1], succ)
        # Caso 3: mesclar os dois filhos e remover recursivamente
        else:
            self._merge(node, index)
            self._remove(node.children[index], key)

    def _remove_child(self, node, key, index):
        child = node.children[index]

        # se deficiente, corrigir antes de descer
        if child.is_deficient():
            self._fix_child(node, index)
            # após fix pode mudar a estrutura; reavaliar index:
            # se index agora é válido e key maior que node.keys[index] então desce para index+1
            if index < len(node.keys) and key > node.keys[index]:
                index += 1

        # desce para o filho apropriado
        self._remove(node.children[index], key)

    ############### FIXING DEFICIENT CHILD ###############
    def _fix_child(self, parent, index):
        child = parent.children[index]

        # emprestar do irmão esquerdo
        if index > 0 and len(parent.children[index - 1].keys) >= 3:
            left = parent.children[index - 1]
            # mover chave do parent para front do child
            child.keys.insert(0, parent.keys[index - 1])
            # mover última chave do left para o parent
            parent.keys[index - 1] = left.keys.pop()
            if not left.is_leaf():
                child.children.insert(0, left.children.pop())
            return

        # emprestar do irmão direito
        if index < len(parent.children) - 1 and len(parent.children[index + 1].keys) >= 3:
            right = parent.children[index + 1]
            # mover chave do parent para o fim do child
            child.keys.append(parent.keys[index])
            # mover primeira chave do right para o parent
            parent.keys[index] = right.keys.pop(0)
            if not right.is_leaf():
                child.children.append(right.children.pop(0))
            return

        # caso nenhum irmão tenha chaves suficientes -> mesclar
        if index > 0:
            self._merge(parent, index - 1)
        else:
            self._merge(parent, index)

    def _merge(self, parent, index):
        left = parent.children[index]
        right = parent.children[index + 1]

        merge_key = parent.keys.pop(index)
        # remover referência ao filho direito
        parent.children.pop(index + 1)

        # combinar
        left.keys.append(merge_key)
        left.keys.extend(right.keys)
        left.children.extend(right.children)

    def _get_min(self, node):
        while not node.is_leaf():
            node = node.children[0]
        return node.keys[0]

    def _get_max(self, node):
        while not node.is_leaf():
            node = node.children[-1]
        return node.keys[-1]

    ############### VALIDATION ###############
    def validate(self):
        return self._validate(self.root, True)

    def _validate(self, node, is_root):
        # checar limites de chaves
        if not is_root:
            if len(node.keys) < 2 or len(node.keys) > 4:
                return False
        else:
            if len(node.keys) > 4:
                return False

        # chaves ordenadas
        if node.keys != sorted(node.keys):
            return False

        # filhos consistentes
        if node.is_leaf():
            return True

        if len(node.children) != len(node.keys) + 1:
            return False

        # validar recursivamente
        for child in node.children:
            if not self._validate(child, False):
                return False

        return True

    ############### ASCII DRAWING ###############
    def draw(self):
        lines = self._draw(self.root)
        for line in lines:
            print(line)

    def _draw(self, node, prefix="", is_last=True):
        lines = []
        connector = "└── " if is_last else "├── "
        lines.append(prefix + connector + str(node.keys))
        prefix += "    " if is_last else "│   "
        for i, child in enumerate(node.children):
            lines.extend(self._draw(child, prefix, i == len(node.children) - 1))
        return lines

    ############### BUSCA SIMPLES (RETORNA ONDE) ###############
    def search(self, key):

      node = self.root

      while True:
          i = 0
          # anda entre as chaves
          while i < len(node.keys) and key > node.keys[i]:
              i += 1

          # encontrado
          if i < len(node.keys) and node.keys[i] == key:
              return True, node.keys.copy()

          # se for folha → acabou
          if node.is_leaf():
              return False, node.keys.copy()


          # garantir que o índice é válido
          if i >= len(node.children):
              # isso nunca deveria acontecer em árvore válida
              return False, node.keys.copy()

          # desce corretamente
          node = node.children[i]

In [ ]:
tree = StrictBTree()
for x in [10,20,30,40,50,60,70,80,90,100,110,120]:
    tree.insert(x)
tree.draw()

print(tree.search(90))   # (True, [...])
print(tree.search(55))   # (False, [...])
print("Valid?", tree.validate())

In [ ]:
tree.remove(20)
tree.remove(80)
tree.draw()
tree.remove(100)
tree.draw()
tree.remove(10)
tree.draw()
tree.remove(60)
tree.draw()

In [ ]:
tree2 = StrictBTree()
for x in range(1, 32):
    tree2.insert(x)

tree2.draw()